In [16]:
pip install duckdb

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import torch
import duckdb


In [ ]:
import duckdb

file_path = r"P:\IIM Ranchi\nasdaq_exteral_data.csv"

duckdb_conn = duckdb.connect()

query = f"""
WITH data AS (
    SELECT
        *
    FROM read_csv(
        '{file_path}',
        AUTO_DETECT=TRUE,
        DELIM=',',
        QUOTE='"',
        ESCAPE='\"',          -- handles messy quotes
        IGNORE_ERRORS=TRUE,   -- skip broken rows
        SAMPLE_SIZE=200000    -- improves schema detection
    )
    WHERE
        Article IS NOT NULL
        AND TRIM(Article) <> ''
        AND Date IS NOT NULL
        AND TRIM(Stock_symbol) <> ''
)
SELECT *
FROM data
USING SAMPLE reservoir(35000);
"""

sampled_df = duckdb_conn.execute(query).df()
sampled_df.to_parquet("sampled_35k_news.parquet")

print("Final sampled rows:", len(sampled_df))


Final sampled rows: 35000


In [14]:
import pandas as pd

df = pd.read_parquet("sampled_35k_news.parquet")

print(len(df))


35000


In [16]:
import pandas as pd

df = pd.read_parquet("sampled_35k_news.parquet")

# Keep essential columns — adjust names as needed
expected_cols = ["Date", "Stock_symbol", "Article"]
df = df[[c for c in expected_cols if c in df.columns]]

# Clean missing/empty articles
df = df.dropna(subset=['Article'])
df = df[df['Article'].str.strip() != ""]

# Parse dates
df['Date'] = pd.to_datetime(df['Date'])

# Sort for consistency
df = df.sort_values(["Stock_symbol", "Date"]).reset_index(drop=True)

df.to_parquet("sampled_35k_clean.parquet")
print(df.head())
print("Cleaned size:", len(df))


        Date Stock_symbol                                            Article
0 2022-11-04            A  Microchip Technology MCHP reported second-quar...
1 2023-01-19            A  Momentum investing is all about the idea of fo...
2 2023-02-23            A  Etsy, Inc. ETSY reported fourth-quarter 2022 e...
3 2023-08-16            A  What you need to know…\nThe S&P 500 Index ($SP...
4 2023-08-25            A  By David Carnevali\nNEW YORK, Aug 25 (Reuters)...
Cleaned size: 35000


In [6]:
pip uninstall keras -y


Found existing installation: keras 3.9.2
Uninstalling keras-3.9.2:
  Successfully uninstalled keras-3.9.2
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install tensorflow==2.15


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow==2.15 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0, 2.19.1, 2.20.0rc0, 2.20.0)

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for tensorflow==2.15


In [9]:
pip install tf-keras


   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------ --------------------- 0.8/1.7 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 5.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/331.9 MB ? eta -:--:--
   ---------------------------------------- 1.3/331.9 MB 6.7 MB/s eta 0:00:50
   ---------------------------------------- 3.1/331.9 MB 7.7 MB/s eta 0:00:43
    --------------------------------------- 5.5/331.9 MB 9.1 MB/s eta 0:00:36
   - -------------------------------------- 8.4/331.9 MB 10.6 MB/s eta 0:00:31
   - -------------------------------------- 11.5/331.9 MB 11.6 MB/s eta 0:00:28
   - -------------------------------------- 15.7/331.9 MB 13.0 MB/s eta 0:00:25
   -- ------------------------------------- 20.4/331.9 MB 14.3 MB/s eta 0:00:22
   --- ------------------------------------ 25.4/331.9 MB 15.6 MB/s eta 0:00:20
   --- ------------------------------------ 32.2/331.9 MB 17.5 MB/s eta 0:0

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flwr 1.15.2 requires protobuf<5.0.0,>=4.21.6, but you have protobuf 6.33.2 which is incompatible.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.33.2 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.33.2 which is incompatible.
mediapipe 0.10.14 requires protobuf<5,>=4.25.3, but you have protobuf 6.33.2 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip insta

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm
import pandas as pd
import torch

df = pd.read_parquet("sampled_35k_clean.parquet")

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load model on GPU
model = SentenceTransformer(
    "sentence-transformers/all-mpnet-base-v2",
    device=device
)

def embed_batch(texts, batch_size=96):  
    all_embs = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        emb = model.encode(
            batch,
            batch_size=batch_size,
            device=device,
            show_progress_bar=False
        )
        all_embs.append(emb)
    return np.vstack(all_embs)

df["embedding"] = list(embed_batch(df["Article"].tolist()))
df.to_parquet("sampled_35k_embedded.parquet")

print("Done embedding!")


c:\Users\Sivapriya\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 365/365 [42:21<00:00,  6.96s/it]  


Done embedding!


In [1]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("Current device:", torch.cuda.current_device() if torch.cuda.is_available() else "None")
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


CUDA available: True
Current device: 0
Device name: NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [4]:
df = pd.read_parquet("sampled_35k_embedded.parquet")

daily_embeddings = (
    df.groupby(["Stock_symbol", "Date"])["embedding"]
      .apply(lambda x: np.mean(np.vstack(x), axis=0))
      .reset_index()
      .rename(columns={"embedding": "E_t"})
)

daily_embeddings.to_parquet("daily_embeddings.parquet")
print("Saved daily embeddings!")


Saved daily embeddings!


In [8]:
import numpy as np
import pandas as pd

alpha = 0.2

daily_embeddings = pd.read_parquet("daily_embeddings.parquet")

def compute_ema(arr, alpha):
    ema = arr[0]
    out = [ema]

    for v in arr[1:]:
        ema = alpha * v + (1 - alpha) * ema
        out.append(ema)

    return np.vstack(out)

records = []

for stock, g in daily_embeddings.groupby("Stock_symbol"):
    g = g.sort_values("Date")

    # THIS was missing
    E_arr = np.vstack(g["E_t"].values)

    mu_arr = compute_ema(E_arr, alpha)

    # shift to ensure mu_{t-1}
    mu_arr = np.vstack([mu_arr[0], mu_arr[:-1]])

    g["mu_t_minus_1"] = list(mu_arr)
    records.append(g)

ssd_base = pd.concat(records, ignore_index=True)
ssd_base.to_parquet("ssd_baseline.parquet")

print("EMA baseline computed correctly (no look-ahead).")


EMA baseline computed correctly (no look-ahead).


In [9]:
from numpy.linalg import norm
import numpy as np

def cosine_distance(a, b):
    return 1 - (np.dot(a, b) / (norm(a) * norm(b)))

ssd_base["SSD"] = [
    cosine_distance(e, mu)
    for e, mu in zip(ssd_base["E_t"], ssd_base["mu_t_minus_1"])
]

ssd_base.to_parquet("ssd_final.parquet")
print("SSD computed!")


SSD computed!
